In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import itertools
import time

In [2]:
class EmbeddingDict(object):
    def __init__(self, path):
        self.word_to_idx = {}
        vecs = []

        for line in open(path, 'r'):
            word, *dims = line.split(' ')
            dims = torch.FloatTensor([float(x) for x in dims])
            self.word_to_idx[word] = len(self.word_to_idx)
            vecs.append(dims)
        self.embeddings = torch.stack(vecs)

    def convert(self, sentence):
        idxs = [self.word_to_idx.get(w, -1) for w in sentence]
        idxs = [i for i in idxs if i >= 0]
        if len(idxs) == 0:
            return None
        return autograd.Variable(torch.LongTensor(idxs))

In [3]:
class GRUTagger(nn.Module):

    def __init__(self, embed_dict, hidden_dim, num_classes):
        super(GRUTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.embed_dict = embed_dict
        vocab_size, embedding_dim = embed_dict.embeddings.size()
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        # freeze embeddings
        self.word_embeddings.weight.data.copy_(embed_dict.embeddings)
        self.word_embeddings.weight.requires_grad = False
        # single layer bidirectional GRU
        self.num_dirs = 2
        self.gru = nn.GRU(embedding_dim, hidden_dim, bidirectional=True)
        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(self.num_dirs * hidden_dim, num_classes)
        self.dropout = nn.Dropout(0.5)
        self.hidden = self.init_hidden

    @property
    def init_hidden(self):
        return autograd.Variable(torch.zeros(self.num_dirs, 1, self.hidden_dim))

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        flatten_input = embeds.view(len(sentence), 1, -1)
        gru_out, self.hidden = self.gru(flatten_input, self.hidden)
        hidden_state = gru_out[-1].view(1, self.num_dirs * self.hidden_dim)
        hidden_state = self.dropout(hidden_state)
        return self.hidden2tag(hidden_state)

In [4]:
def load_data(path):
    for line in open(path, 'rb'):
        try:
            label, *sent = line.decode('utf-8').split(' ')
            yield sent, int(label)
        except:
            continue

In [5]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
    remainder = len(lst) % n
    if remainder > 0:
        yield lst[-remainder:]

In [6]:
def evaluate(model, data):
    model.eval()
    num_correct, num_total = 0, 0
    for sentence, label in data:
        model.hidden = model.init_hidden
        sentence = embed_dict.convert(sentence)
        if sentence is None:
            continue
        tag_scores = model(sentence)
        if tag_scores is None:
            continue
        predict = tag_scores.data.numpy().argmax()
        if predict == label:
            num_correct += 1
        num_total += 1
    return float(num_correct)/float(num_total)

In [7]:
n = 2000
batch_size = 32
lstm_size = 25
embed_dict = EmbeddingDict("data/small-glove.50d.txt")
train_data = list(itertools.islice(load_data("data/sentiment-train10k.txt"), n))
test_data = list(itertools.islice(load_data("data/sentiment-test10k.txt"), n))
model = GRUTagger(embed_dict, lstm_size, 2)
loss_function = nn.CrossEntropyLoss()
learned_params = [param for pname, param in model.named_parameters() if 'word_embeddings' not in pname]
optimizer = optim.Adadelta(learned_params)

Exception ignored in: <generator object load_data at 0x104337d58>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object load_data at 0x104337d58>
RuntimeError: generator ignored GeneratorExit


In [8]:
for epoch in range(10):
    total_loss = torch.Tensor([0])
    start_iter = int(round(time.time() * 1000))
    model.train()
    for batch in chunks(train_data, batch_size):
        model.zero_grad()
        for sentence, label in batch:
            model.hidden = model.init_hidden
            sentence = embed_dict.convert(sentence)
            if sentence is None:
                continue
            tag_scores = model(sentence)
            label_var = autograd.Variable(torch.LongTensor([label]))
            loss = loss_function(tag_scores, label_var)
            total_loss += loss.data
            loss.backward()
        optimizer.step()

    train_acc = evaluate(model, train_data)
    test_acc = evaluate(model, test_data)
    print('Train Accuracy: ', train_acc)
    print('Test Accuracy: ', test_acc)
    num_millis = int(round(time.time() * 1000)) - start_iter
    print('End of epoch {}: {} [{} ms]'.format(epoch, total_loss[0], num_millis))

Train Accuracy:  0.599799899949975
Test Accuracy:  0.5925
End of epoch 0: 1381.95458984375 [31172 ms]
Train Accuracy:  0.7013506753376688
Test Accuracy:  0.688
End of epoch 1: 1273.088623046875 [28313 ms]
Train Accuracy:  0.7248624312156078
Test Accuracy:  0.7125
End of epoch 2: 1157.6312255859375 [30121 ms]
Train Accuracy:  0.7373686843421711
Test Accuracy:  0.723
End of epoch 3: 1090.34033203125 [29576 ms]
Train Accuracy:  0.7528764382191095
Test Accuracy:  0.7345
End of epoch 4: 1042.017333984375 [31145 ms]
Train Accuracy:  0.7673836918459229
Test Accuracy:  0.746
End of epoch 5: 1007.1766967773438 [33939 ms]
Train Accuracy:  0.7748874437218609
Test Accuracy:  0.7515
End of epoch 6: 976.752197265625 [31245 ms]
Train Accuracy:  0.7863931965982992
Test Accuracy:  0.761
End of epoch 7: 943.0791625976562 [29606 ms]
Train Accuracy:  0.7853926963481741
Test Accuracy:  0.764
End of epoch 8: 926.1249389648438 [30087 ms]
Train Accuracy:  0.799399699849925
Test Accuracy:  0.775
End of epoch 9